# Midterm Project: ETL and Data Warehousing
Madelyn Khoury (mgk5ybb) and Tiara Allard

DS 2002 Spring 2023

### Design and Strategy

We chose to model bank transactions as the core business process of our data warehouse, so we designed a database schema centered around bank transactions. To see the schema we designed for this project, please look at the ReadMe of our GitHub project. 

Our schema stores information about bank transactions, bank accounts and users involved in transactions, transaction dates, and transaction locations. We were unable to find a database/dataset with all this information, so instead we combined data from multiple different data sources. This had the added benefit of allowing us to meet the requirements for importing data from a number of sources.

We combined several dummy/randomly generated datasets to build a complete data warehouse. First, we got bank transaction and account information from a .csv file stored on our local filesystem. Then, we generated user data from an API and linked it to the accounts and transactions. Finally, we imported location information from the Northwind MySQL database to represent regions in which banking transactions might have occurred.

After processing the data and computing useful fields, we formatted it into our fact and dimension tables in the final data warehouse.

### Imports and Helper Functions

In [ ]:
import sys
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install mysql-connector-python
!{sys.executable} -m pip install pymysql
!{sys.executable} -m pip install sqlalchemy
!{sys.executable} -m pip install uszipcode

In [83]:
import datetime
import json
import mysql.connector
import os
import pandas as pd
import pymysql
import requests
from sqlalchemy import create_engine
from uszipcode import SearchEngine, SimpleZipcode

In [3]:
def get_api_response(url, headers, params, response_type):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        # result = json.dumps(response.json(), sort_keys=True, indent=4)
        result = response.json()
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"
        
    return result

In [4]:
# this helper function is inspired by part of one of the provided files, 02-Python-MySQL.ipynb
def get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    dframe = None
    try:
        conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
        sqlEngine = create_engine(conn_str, pool_recycle=3600)
        connection = sqlEngine.connect()
        try:
            dframe = pd.read_sql(sql_query, connection);
        except:
            print("Sequel query was unsuccessful.")
        connection.close()
        return dframe
    except:
        print("Unable to connect to the MySQL database.")
    return None

In [47]:
# this code snippet is modified from: https://www.geeksforgeeks.org/python-program-to-calculate-age-in-year/ 
def calculate_age(birth_date):
    birth_date = datetime.datetime.strptime(birth_date, '%Y-%m-%d').date()
    today = datetime.date.today()
    try:
        birthday = birth_date.replace(year = today.year)
 
    # raised when birth date is February 29 but it's not a leap year
    except ValueError:
        birthday = birth_date.replace(year = today.year,
                  month = birth_date.month + 1, day = 1) # birth date becomes march 1st
 
    if birthday > today:
        return today.year - birth_date.year - 1
    else:
        return today.year - birth_date.year

In [64]:
def get_region(state_abbreviation):
    if state_abbreviation in {"WA", "OR", "CA", "ID", "MT", "NV", "UT", "CO", "WY", "AK"}:
        return "West"
    elif state_abbreviation in {"AZ", "NM", "TX", "OK"}:
        return "Southwest"
    elif state_abbreviation in {"ND", "SD", "NE", "KS", "MN", "IA", "MO", "WI", "IL", "MI", "OH"}:
        return "Midwest"
    elif state_abbreviation in {"ME", "NH", "MA", "CT", "RI", "VT", "NY", "PA", "DE", "MD", "NJ"}:
        return "Northeast"
    else:
        return "Southeast"

In [93]:
def get_zipcode(city, state):
    search = SearchEngine()
    results = search.by_city_and_state(city, state)
    if len(results) > 0:
        return results[0].zipcode
    else:
        return None

### Importing Data From Local File System

The core bank transaction information that we will use came from a dataset on Kaggle (https://www.kaggle.com/datasets/apoorvwatsky/bank-transaction-data). We downloaded the data in the form of a csv file and will import it from the local filesystem in order to be used in our data warehouse.

In [14]:
bank_info_path = os.path.join(os.getcwd(), 'bank.xlsx')
bank_info = pd.read_excel(bank_info_path)

In [15]:
bank_info

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1.000000e+06,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2.000000e+06,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2.500000e+06,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5.500000e+06,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6.000000e+06,.
...,...,...,...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,NaN,2019-03-05,117934.30,NaN,-1.901902e+09,.
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901602e+09,.
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901302e+09,.
116199,409000362497',2019-03-05,IMPS 05-03-20194C,NaN,2019-03-05,109868.65,NaN,-1.901412e+09,.


In [16]:
account_info = bank_info[["Account No", "BALANCE AMT"]]
account_info

,Account No,BALANCE AMT
0,409000611074',1.000000e+06
1,409000611074',2.000000e+06
2,409000611074',2.500000e+06
3,409000611074',5.500000e+06
4,409000611074',6.000000e+06
...,...,...
116196,409000362497',-1.901902e+09
116197,409000362497',-1.901602e+09
116198,409000362497',-1.901302e+09
116199,409000362497',-1.901412e+09


The bank transaction fact table can store the new balance of the pertinent account after said transaction, but we want the bank account table to store current info about each account. So, we will take the most recent balance for each account and create a "Current Balance" field in the accounts table.  Since the rows of the spreadsheet were sorted in order of transaction date, then the most recent balance for each account will be the balance in the last-occurring transaction.

In [13]:
account_info = account_info.drop_duplicates(subset=["Account No"], keep="last") # keep only the last record for each account
account_info

,Account No,BALANCE AMT
1092,409000611074',4.622000e+05
2136,409000493201',7.435833e+05
2938,409000425051',-3.567348e+08
2989,409000405747',-5.482675e+08
7577,409000438611',-5.479193e+08
13591,409000493210',-5.463146e+08
27045,409000438620',-5.399631e+08
37581,1196711',-1.586916e+09
86360,1196428',-1.687234e+09
116200,409000362497',-1.901417e+09


Our schema design includes two other fields in the accounts table: first, the ID of the customer who holds this account, and secondly, the date when the account was created. We will randomly select user IDs from our users table to be the user IDs, and we will randomly generate dates to serve as the date created.

### Importing Data From API

We've chosen to use the `users` endpoint from random-data-api.com, which randomly generates data for a set of users. This will populate the Users table in our data warehouse.

In [35]:
size = 20 # only get info on 20 users for now
url = "https://random-data-api.com/api/v2/users"
querystring = {"size":size}
headers = None

# Get information from users API endpoint
users = get_api_response(url, headers, querystring, "dataframe")
users

,id,uid,password,first_name,last_name,username,email,avatar,gender,phone_number,...,address.zip_code,address.state,address.country,address.coordinates.lat,address.coordinates.lng,credit_card.cc_number,subscription.plan,subscription.status,subscription.payment_method,subscription.term
0,3681,5d390665-f0e8-43b5-93a0-660130255924,Uab8qHoREO,Geraldo,Hayes,geraldo.hayes,geraldo.hayes@email.com,https://robohash.org/voluptasaccusantiumqui.pn...,Female,+27 1-920-927-3622 x009,...,76203-6779,Tennessee,United States,40.840343,-145.342207,4848-8129-4606-0569,Platinum,Active,Bitcoins,Monthly
1,3144,3a5fff0f-42e7-46bc-9436-4464461311bd,eVLIFzasU2,Rocky,Welch,rocky.welch,rocky.welch@email.com,https://robohash.org/nisirerumconsequuntur.png...,Male,+81 (619) 868-3641 x28615,...,35714-5109,California,United States,22.190077,140.126099,6771-8934-0492-3379,Premium,Blocked,Cheque,Annual
2,1204,3c4df319-6599-4a24-ae7b-1515de5a65a8,RD8fOXIT5l,Edward,Leannon,edward.leannon,edward.leannon@email.com,https://robohash.org/consequunturquiperspiciat...,Agender,+265 (451) 156-7427,...,72967,Connecticut,United States,-16.996777,-145.592462,6771-8998-7355-8455,Silver,Pending,Apple Pay,Full subscription
3,878,e6e2a1e3-0d6a-4972-9224-f1c7d8e851e4,fNSYv3h1TZ,Elaina,Olson,elaina.olson,elaina.olson@email.com,https://robohash.org/repellenduseligendiest.pn...,Non-binary,+268 292.383.9385 x030,...,96625,Wyoming,United States,19.989864,49.822155,5129-6831-6263-5351,Silver,Blocked,WeChat Pay,Payment in advance
4,3391,e2f751cd-f51c-4894-a48f-f898ee4cf6b2,1K5tzQX0pw,Deonna,Becker,deonna.becker,deonna.becker@email.com,https://robohash.org/consequaturaplaceat.png?s...,Polygender,+966 1-601-457-1862 x474,...,99232-9994,New Hampshire,United States,37.399301,-177.140826,5492-4414-0374-2547,Professional,Active,Bitcoins,Full subscription
5,5207,ec1b0300-3a2b-4ba5-8961-234120802dfe,q0ZJSf6j4a,Marlon,Schultz,marlon.schultz,marlon.schultz@email.com,https://robohash.org/aspernaturquimagnam.png?s...,Agender,+95 341.366.7782 x272,...,19492-3684,Illinois,United States,20.111573,78.788755,4208-4114-9702-5347,Free Trial,Blocked,Bitcoins,Payment in advance
6,3660,1f8fe1d0-3483-4c6f-b003-dc90c5c6ea40,2EAg8cLO3i,Thurman,Hilpert,thurman.hilpert,thurman.hilpert@email.com,https://robohash.org/autemaliquidunde.png?size...,Non-binary,+290 643-904-5413 x93396,...,17106,New Hampshire,United States,54.995086,153.133955,6771-8935-6952-3113,Silver,Active,Credit card,Annual
7,9397,17f22b41-0633-4eba-adf9-4d6e505be125,y0azQBxgOq,Cheree,Hodkiewicz,cheree.hodkiewicz,cheree.hodkiewicz@email.com,https://robohash.org/modicumrecusandae.png?siz...,Female,+66 359-297-8779,...,25321,Louisiana,United States,58.599366,48.379522,6771-8957-3583-0496,Platinum,Active,WeChat Pay,Payment in advance
8,3963,313e92ef-a44b-4c8c-b036-c15c5ed14fce,pZM64hHwOT,Thanh,Stehr,thanh.stehr,thanh.stehr@email.com,https://robohash.org/consequaturconsequuntursi...,Polygender,+299 622-875-9316 x40901,...,60138,North Carolina,United States,68.969695,145.108755,5479-2069-3208-5503,Diamond,Active,Credit card,Payment in advance
9,5068,9cfd5a51-a4be-4248-a954-123f785510b4,jfJbEzM78F,Suzanne,Cummings,suzanne.cummings,suzanne.cummings@email.com,https://robohash.org/maioresquivoluptates.png?...,Bigender,+374 383.538.9394,...,43950,Colorado,United States,-18.375303,-60.864127,4364251708638,Bronze,Blocked,Cash,Payment in advance


### Importing Data From Relational Database

To get location data that could represent locations in which bank transactions were completed, we've decided to import information about the shipping location of orders from the `orders` table in the Northwind database. In our data warehouse, this information will represent the location in which a customer instigated a banking transaction; perhaps it could represent the location of physical branches of the bank.

In [100]:
# define variables to set up connection to mySQL database
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "northwind"

First we must get the location-related data from the `orders` table.

In [101]:
sql_query = """
    SELECT ship_address, ship_city, ship_state_province, ship_zip_postal_code, ship_country_region from orders;
"""

In [102]:
locations_info = get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query)

### Transforming the Location Data

We got location info from the Northwind database, but we must remove duplicate values so that we have a table of unique locations.

In [103]:
locations_info = locations_info.drop_duplicates()

It appears that the Northwind database didn't store actual zip codes, but instead put 99999 in for every row. So, we will fill in the table with the correct zip code for each city listed. Additionally, we will add another column to the table which will identify the region of the United States that the location is in.

In [99]:
locations_info["zipcode"] = locations_info.apply(lambda row: get_zipcode(row["ship_city"], row["ship_state_province"]), axis=1)
locations_info.drop(["ship_zip_postal_code"], axis = 1, inplace = True)
locations_info

AttributeError: 'NoneType' object has no attribute 'apply'

In [97]:
locations_info["region"] = locations_info.apply(lambda row: get_region(row["ship_state_province"]), axis=1)
locations_info

AttributeError: 'NoneType' object has no attribute 'apply'

### Transforming the User Data

To transform the user data, all we had to do was drop some columns of the dataframe. As we can see by looking at the columns of users, there is a lot of superfluous information.

In [36]:
users.columns

Index(['id', 'uid', 'password', 'first_name', 'last_name', 'username', 'email',
       'avatar', 'gender', 'phone_number', 'social_insurance_number',
       'date_of_birth', 'employment.title', 'employment.key_skill',
       'address.city', 'address.street_name', 'address.street_address',
       'address.zip_code', 'address.state', 'address.country',
       'address.coordinates.lat', 'address.coordinates.lng',
       'credit_card.cc_number', 'subscription.plan', 'subscription.status',
       'subscription.payment_method', 'subscription.term'],
      dtype='object')

In [ ]:
users.drop(['employment.title', 'employment.key_skill', 'uid','avatar', 'social_insurance_number', 'subscription.plan', 'subscription.payment_method', 'subscription.status', 'subscription.term', 'address.city', 'address.street_name', 'address.street_address', 'address.zip_code', 'address.state', 'address.country', 'address.coordinates.lat', 'address.coordinates.lng'], axis = 1, inplace = True)

We will also calculate the age of each user, that way we have calculations stored in our OLAP database and don't have to compute them on the fly.

In [51]:
users["age"] = users.apply(lambda row: calculate_age(row["date_of_birth"]), axis=1)

In [52]:
users

,id,password,first_name,last_name,username,email,gender,phone_number,date_of_birth,credit_card.cc_number,age
0,3681,Uab8qHoREO,Geraldo,Hayes,geraldo.hayes,geraldo.hayes@email.com,Female,+27 1-920-927-3622 x009,1990-03-10,4848-8129-4606-0569,33
1,3144,eVLIFzasU2,Rocky,Welch,rocky.welch,rocky.welch@email.com,Male,+81 (619) 868-3641 x28615,1973-09-26,6771-8934-0492-3379,49
2,1204,RD8fOXIT5l,Edward,Leannon,edward.leannon,edward.leannon@email.com,Agender,+265 (451) 156-7427,1978-11-26,6771-8998-7355-8455,44
3,878,fNSYv3h1TZ,Elaina,Olson,elaina.olson,elaina.olson@email.com,Non-binary,+268 292.383.9385 x030,2001-06-19,5129-6831-6263-5351,21
4,3391,1K5tzQX0pw,Deonna,Becker,deonna.becker,deonna.becker@email.com,Polygender,+966 1-601-457-1862 x474,1993-10-06,5492-4414-0374-2547,29
5,5207,q0ZJSf6j4a,Marlon,Schultz,marlon.schultz,marlon.schultz@email.com,Agender,+95 341.366.7782 x272,1990-04-06,4208-4114-9702-5347,32
6,3660,2EAg8cLO3i,Thurman,Hilpert,thurman.hilpert,thurman.hilpert@email.com,Non-binary,+290 643-904-5413 x93396,1975-10-30,6771-8935-6952-3113,47
7,9397,y0azQBxgOq,Cheree,Hodkiewicz,cheree.hodkiewicz,cheree.hodkiewicz@email.com,Female,+66 359-297-8779,1983-01-19,6771-8957-3583-0496,40
8,3963,pZM64hHwOT,Thanh,Stehr,thanh.stehr,thanh.stehr@email.com,Polygender,+299 622-875-9316 x40901,1975-03-02,5479-2069-3208-5503,48
9,5068,jfJbEzM78F,Suzanne,Cummings,suzanne.cummings,suzanne.cummings@email.com,Bigender,+374 383.538.9394,1992-04-29,4364251708638,30
